In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import sqlite3 as db
import datetime as dt
#from sklearn.metrics import mean_squared_error

In [4]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [5]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [6]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings

LSTM analysis of common birds

In [7]:
commonCore = birdList_request(StopName='EastSunsetWay',cnx=connectDB())
commonCore = commonCore[commonCore.apply(lambda x: (x['stopGroup']==4) and (x['overall']==4),axis=1)]

In [8]:
querySpecies = 'amerob'
StopName = 'EastSunsetWay'
query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode = "{querySpecies}" AND StopName = "{StopName}";'
obsData = pd.read_sql(query,con=connectDB(),parse_dates=['obsDt'])
obsData.set_index('obsDt',inplace=True)

In [82]:
obsDataDaily = obsData.resample('d')
dailyData = obsDataDaily.mean()
dailyData

,howMany
obsDt,
2019-03-23,5.0
2019-03-24,NaN
2019-03-25,NaN
2019-03-26,NaN
2019-03-27,NaN
...,...
2022-03-06,NaN
2022-03-07,NaN
2022-03-08,NaN


In [ ]:
def eval_forecasts(actual,predicted):
    try:
        scores = list()
        for i in range(actual.shape[1]):
            mse = mean_squared_error(actual[:,i],predicted[:,i])
            rmse = np.sqrt(mse)
            scores.append(rmse)
        s = 0
        for row in range(actual.shape[0]):
            for col in range(actual.shapre[1]):
                s+= (actual[row,col] - predicted[row,col]**2)
        score = np.sqrt(s/(actual.shape[0]*actual.shapre[1]))
    except Exception as metricExc:
        raise metricExc
    return score, scores

In [83]:
fdd = dailyData['howMany'].fillna(method='pad')
fdd = pd.DataFrame(fdd)
fdd

,howMany
obsDt,
2019-03-23,5.0
2019-03-24,5.0
2019-03-25,5.0
2019-03-26,5.0
2019-03-27,5.0
...,...
2022-03-06,11.0
2022-03-07,11.0
2022-03-08,11.0


In [43]:
def split_trainTest(dailyObs):
    train, test = dailyObs[1:-327],dailyObs[-328:-6]
    train = np.array(np.split(train,len(train)/7))
    test = np.array(np.split(test,len(test)/7))
    return train, test

In [81]:
train, test = split_trainTest(dailyObs=fdd)
print(train.shape)
print(test.shape)

(108, 7, 1)
(46, 7, 1)


In [120]:
#build forward-moving windows
def to_supervised(train,nInput,nOut = 7):
    train = train.reshape((train.shape[0]*train.shape[1],train.shape[2]))
    X, y = [],[]
    inStart = 0
    for _ in range(len(train)):
        inEnd = inStart + nInput
        outEnd = inEnd + nOut
        if outEnd <= len(train):
            xIn = train[inStart:inEnd,0]
            xIn = xIn.reshape((len(xIn),1))
            X.append(xIn)
            y.append(train[inEnd:outEnd,0])
        inStart+= 1
    return np.array(X),np.array(y)

In [121]:
X, y = to_supervised(train=train,nInput=7)
print(X.shape)
print(y.shape)

(743, 7, 1)
(743, 7)


In [123]:
def buildModel(train,nInput):
    train_x,train_y = to_supervised(train,nInput)
    verbose, epochs, batch_size = 0,70,16
    nTimesteps,nFeatures,nOutputs = train_x.shape[1],train_x.shape[2],train_y.shape[1]
    model = Sequential()
    model.add(LSTM(units=200,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(nOutputs))
    model.compile(loss='mse',optimizer='adam')
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose)
    return model

In [124]:
buildModel(train=train,nInput=7)

In [125]:
def forecast(model,history,nInput):
    histData = np.array(history)
    histData = histData.reshape((histData.shape[0]*histData.shape[1],histData.shape[2]))
    inputX = histData[-nInput:,0]
    inputX = inputX.reshape((1,len(inputX),1))
    yhat = model.predict(inputX,verbose=0)
    yhat = yhat[0]
    return yhat

In [126]:
history = [x for x in train]
history

[array([[5.],
        [5.],
        [5.],
        [5.],
        [3.],
        [3.],
        [3.]]),
 array([[ 1.],
        [ 1.],
        [ 1.],
        [47.],
        [47.],
        [47.],
        [47.]]),
 array([[47.],
        [47.],
        [47.],
        [47.],
        [47.],
        [47.],
        [47.]]),
 array([[47.],
        [47.],
        [47.],
        [47.],
        [47.],
        [47.],
        [47.]]),
 array([[47.],
        [47.],
        [47.],
        [ 4.],
        [ 4.],
        [ 4.],
        [ 4.]]),
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]),
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]),
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]),
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]),
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
  